# Softmax Word2Vec method

> About the autor: Jinming Yang is currently an undergraduate school student at Sun Yet-san University who focuses on transportation research and machine learning.

For a 10000 word vocabulary, using one-hot vectors as the representation of each word in a neural network is computational expensive. Besides, one-hot vectors can not represent the correlations between words like "Soviet" and "Union", "United" and "States"... So we need a new representation of those words. These new representations of words have much lower dimension like 128. The correlations between strong connected words can be reflected in their corresponding representations.

In order to do that, **Word2Vec** was introduced to derive those word representations. Hereby, we'll mainly focuse on the **Softmax Word2Vec method**.

## Methodology
Basically the Softmax Word2Vec method use an three layer autoencoder neural network to derive word representations. The first layer is the input layer consists of 10000 neurons(units). The second layer is the hidden layer which consists of 128 neurons(no activation function). The third layer is the softmax layer which consists of 10000 neurons.

**Network setting**

|`Network Layer`  |`Number of neurons`|`Activation`|
|:----------------|------------------:|-----------:|
|**Input layer**  |     **10000**     |  **None**  |
|**Hidden layer** |      **128**      |  **None**  |
|**Output layer** |     **10000**     |**Sigmoid** |

In [1]:
import numpy as np
import tensorflow as tf
import collections
import math

### Preparing the corpus
Downloading the corpus in "mattmahoney.net/dc/text8.zip". For researchers in main land China the website won't be available. You can also download it in my GitHub repositry "Learning Machine Learning".

In [2]:
#Read the file
f = open("ptbdata/ptb.train.txt","r")
rawData = f.read()
print(rawData[:1000])

 aer banknote berlitz calloway centrust cluett fromstein gitano guterman hydro-quebec ipo kia memotec mlx nahb punts rake regatta rubens sim snack-food ssangyong swapo wachter 
 pierre <unk> N years old will join the board as a nonexecutive director nov. N 
 mr. <unk> is chairman of <unk> n.v. the dutch publishing group 
 rudolph <unk> N years old and former chairman of consolidated gold fields plc was named a nonexecutive director of this british industrial conglomerate 
 a form of asbestos once used to make kent cigarette filters has caused a high percentage of cancer deaths among a group of workers exposed to it more than N years ago researchers reported 
 the asbestos fiber <unk> is unusually <unk> once it enters the <unk> with even brief exposures to it causing symptoms that show up decades later researchers said 
 <unk> inc. the unit of new york-based <unk> corp. that makes kent cigarettes stopped using <unk> in its <unk> cigarette filters in N 
 although preliminary findings wer

After opening the file, we can transfer the content in the file to string format. Then we can split the text string to individual words by blank.

In [3]:
# Transfer the raw data as strings using Tensorflow. 
# Then split it into individual words
dataStr = tf.compat.as_str(rawData) #Convert to string
print('Split:')
data = dataStr.split() #Split by blank
print(data[:100])
print(len(data))

Split:
['aer', 'banknote', 'berlitz', 'calloway', 'centrust', 'cluett', 'fromstein', 'gitano', 'guterman', 'hydro-quebec', 'ipo', 'kia', 'memotec', 'mlx', 'nahb', 'punts', 'rake', 'regatta', 'rubens', 'sim', 'snack-food', 'ssangyong', 'swapo', 'wachter', 'pierre', '<unk>', 'N', 'years', 'old', 'will', 'join', 'the', 'board', 'as', 'a', 'nonexecutive', 'director', 'nov.', 'N', 'mr.', '<unk>', 'is', 'chairman', 'of', '<unk>', 'n.v.', 'the', 'dutch', 'publishing', 'group', 'rudolph', '<unk>', 'N', 'years', 'old', 'and', 'former', 'chairman', 'of', 'consolidated', 'gold', 'fields', 'plc', 'was', 'named', 'a', 'nonexecutive', 'director', 'of', 'this', 'british', 'industrial', 'conglomerate', 'a', 'form', 'of', 'asbestos', 'once', 'used', 'to', 'make', 'kent', 'cigarette', 'filters', 'has', 'caused', 'a', 'high', 'percentage', 'of', 'cancer', 'deaths', 'among', 'a', 'group', 'of', 'workers', 'exposed', 'to', 'it']
887521


### Create dictionary and numerical representation of the corpus
We have the words sequence in the corpus. Now we want to use numbers to represent each word and create the dictionary of words and their corresponding number. After that, we can convert the whole text from word sequence to number sequence.

#### Count word number
First, count the number of occurence of each word in the corpus.

In [4]:
counted_words = collections.Counter(data)  #counted_words is a dictionary {'word1': num_1, 'word2': num_2, ...}
print('%d different words were found in the corpus'%(len(counted_words)))
print()
print('Part of the dictionary:')
dict(list(counted_words.items())[0:10])    #print 10 of them

9999 different words were found in the corpus

Part of the dictionary:


{'aer': 1,
 'banknote': 1,
 'berlitz': 1,
 'calloway': 1,
 'centrust': 1,
 'cluett': 1,
 'fromstein': 1,
 'gitano': 1,
 'guterman': 1,
 'hydro-quebec': 1}

#### Select top frequent words
From above, we can see that there are 253854 diffenent words in the corpus. However most of them have very few occurence, building a dictionary for them is highly uneconomical and inefficient. So we just choose the top 9999 frequent words to build the dictionary. The rest of the words which are less likely to occur were classified as 'LFW' aka 'Low Frequency Words'.

In [5]:
freq_counted_words = dict(counted_words.most_common(9999))
print('There are %d words in the selected dictionary'%(len(freq_counted_words)))

There are 9999 words in the selected dictionary


Now conbine all other less frequently occured words as 'LFW', count their number of occurence.

In [6]:
lfw_count = 0
for word in counted_words:
    if not (word in freq_counted_words.keys()):
        lfw_count += 1
word_count_dict = {'lfw': lfw_count}
word_count_dict.update(freq_counted_words)
print('There are %d words in the dictionary'%(len(word_count_dict)))
print()
print('The first 5 entries in the dictionary is:')
dict(list(word_count_dict.items())[:5])

There are 10000 words in the dictionary

The first 5 entries in the dictionary is:


{'lfw': 0, 'the': 50770, '<unk>': 45020, 'N': 32481, 'of': 24400}

We have already selected the top 10000 frequent words in the corpus. Now we need to index them with numbers aka establish the word2number projection.

In [7]:
ind = 0
word_dict = {}
for word in word_count_dict:
    word_dict.update({word: ind})
    ind += 1
print('The first 5 entries in the dictionary "word_dict" is: ')
dict(list(word_dict.items())[:5])

The first 5 entries in the dictionary "word_dict" is: 


{'lfw': 0, 'the': 1, '<unk>': 2, 'N': 3, 'of': 4}

Since we have already built the dictionary of words and their corresponding number(index). We can now transfer the initial corpus to a number sequence.

In [8]:
num_corpus = []
for word in data:
    if word in word_dict.keys():
        num_corpus.append(word_dict[word])
    else:
        num_corpus.append(word_dict['lfw'])
print('The first 100 words in the corpus represented by their corresponding indeces are:')
print(num_corpus[:100])

The first 100 words in the corpus represented by their corresponding indeces are:
[9970, 9971, 9972, 9973, 9974, 9975, 9976, 9977, 9978, 9979, 9980, 9981, 9982, 9983, 9984, 9985, 9986, 9987, 9988, 9989, 9990, 9991, 9992, 9993, 8569, 2, 3, 72, 393, 33, 2116, 1, 146, 19, 6, 8570, 275, 407, 3, 23, 2, 13, 141, 4, 2, 5278, 1, 3055, 1581, 96, 7232, 2, 3, 72, 393, 8, 337, 141, 4, 2468, 657, 2158, 949, 24, 521, 6, 8570, 275, 4, 39, 303, 437, 3661, 6, 941, 4, 3143, 495, 262, 5, 137, 5882, 4219, 5883, 30, 986, 6, 240, 755, 4, 1013, 2765, 211, 6, 96, 4, 427, 4060, 5, 14]


We have the dictionary to project words to indeces. We now have to build a dictionary to project each index to its corresponding word. So that we can recover text from index sequence.

In [9]:
index_dict = dict(zip(word_dict.values(), word_dict.keys()))
print('The first 10 entries in dictionary index_dict is: ')
dict(list(index_dict.items())[:10])

The first 10 entries in dictionary index_dict is: 


{0: 'lfw',
 1: 'the',
 2: '<unk>',
 3: 'N',
 4: 'of',
 5: 'to',
 6: 'a',
 7: 'in',
 8: 'and',
 9: "'s"}

## Skip-gram

The **skip-gram** conscept is used to create batches for the autoencoder neural network. It basically has two main conscepts:

* **gram**: **gram** is a fix-sized sliding window over the corpus.

* **skip**: **skip** is the number of times a word repeated in the dataset with different context words.

The central word of the gram is called the **target**. And the rest of the words in the gram is the context words of the **target**. Utilising every context word may be computational expensive, thus we just randomly choose **skip** different context words for one **target** to train the autoencoder neural network.

### Neural network inputs and their labels

Based on the conscept above, the input of the neural network is the initial representation of each **target** word. For example, in this experiment setting, the input would be a 10000 dimension one-hot vector denoting each **target**. And the label would also be a 10000 dimension one-hot vector denoting one of the context word of that  **target**. Basically, the inputs and their label are this kind of combination: 

${[target_1, context_1^{(i)}], [target_1, context_2^{(i)}], ..., [target_1, context_{skip}^{(i)}], ..., [target_{ng}, context_1^{(i)}], [target_{ng}, context_2^{(i)}], [target_{ng}, context_{skip}^{(i)}]}$.

<img src="images/skip_gram.png" alt="Drawing" style="width: 500px;"/>

The following function can generate a mini-batch with size **batch_size**. **$target$** and **context** were generated separatley, both of which have the length of **batch_size**.

In [10]:
data_index = 0
# generate batch data
def generate_batch(data, batch_size, skip, sub_gram):
    global data_index
    assert batch_size % skip == 0
    assert skip <= 2 * sub_gram
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    context = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * sub_gram + 1  # [ sub_gram input_word sub_gram]
    buffer = collections.deque(maxlen=span)
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    for i in range(batch_size // skip):
        target = sub_gram  # input word at the center of the buffer
        targets_to_avoid = [sub_gram]
        for j in range(skip):
            while target in targets_to_avoid:
                target = np.random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i * skip + j] = buffer[sub_gram]  # this is the input word
            context[i * skip + j, 0] = buffer[target]  # these are the context words
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    # Backtrack a little bit to avoid skipping words in the end of a batch
    data_index = (data_index + len(data) - span) % len(data)
    return batch, context

## Autoencoder Network Setting
Neural network settings:

In [11]:
vocabulary_size = 10000
batch_size = 128
embedding_size = 128  # Dimension of the embedding vector aka the number of units in the hidden layer.
sub_gram = 2          # (Span_of_gram-1)/2
skip = 2              # How many times to reuse an input to generate a context.

Set placeholders for inputs and outputs:

In [12]:
train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
train_context = tf.placeholder(tf.int32, shape=[batch_size, 1])

Set variables in the network: the embedding here is a $(10000\times128)$ weight matrix

In [13]:
# weight matrix between input layer and hidden layer
embeddings = tf.Variable(
    tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))

# get the corresponding embedding of input word.
embed = tf.nn.embedding_lookup(embeddings, train_inputs)

# weight matrix between hidden layer and the softmax layer
weights = tf.Variable(tf.truncated_normal([embedding_size, vocabulary_size],
                          stddev=1.0 / math.sqrt(embedding_size)))

# biases of softmax layer
biases = tf.Variable(tf.zeros([vocabulary_size]))

hidden_out = tf.matmul(embed, weights) + biases

Instructions for updating:
Colocations handled automatically by placer.


Convert the context into a one-hot format:

In [14]:
train_one_hot = tf.one_hot(train_context, vocabulary_size)

Using crossentropy as the loss function.

In [15]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hidden_out, 
    labels=train_one_hot))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



Set a gradient descent minimizer for the **cross_entropy**.

In [16]:
# Construct the SGD optimizer using a learning rate of 1.0.
optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(cross_entropy)

Instructions for updating:
Use tf.cast instead.


Initialize all the parameters.

In [17]:
init = tf.global_variables_initializer()

### Neural Network training

In [21]:
num_steps = 10000
with tf.Session() as session:
  # We must initialize all variables before we use them.
  init.run()
  print('Initialized')

  average_loss = 0
  for step in range(num_steps):
    batch_inputs, batch_context = generate_batch(num_corpus,
        batch_size, skip, sub_gram)
    feed_dict = {train_inputs: batch_inputs, train_context: batch_context}

    # We perform one update step by evaluating the optimizer op (including it
    # in the list of returned values for session.run()
    _, loss_val = session.run([optimizer, cross_entropy], feed_dict=feed_dict)
    average_loss += loss_val

    if (step + 1) % 500 == 0:
        average_loss /= 500
        # The average loss is an estimate of the loss over the last 2000 batches.
        print('Average loss at step ', step + 1, ': ', average_loss)
        average_loss = 0

Initialized
Average loss at step  500 :  7.679187176704406
Average loss at step  1000 :  7.100021776199341
Average loss at step  1500 :  6.862011333465576
Average loss at step  2000 :  6.782251722335816
Average loss at step  2500 :  6.570901042938233
Average loss at step  3000 :  6.7042559719085695
Average loss at step  3500 :  6.549054152488709
Average loss at step  4000 :  6.45622038936615
Average loss at step  4500 :  6.593372061729431
Average loss at step  5000 :  6.632398215293884
Average loss at step  5500 :  6.549637966156006
Average loss at step  6000 :  6.408308345794678
Average loss at step  6500 :  6.5721294832229615
Average loss at step  7000 :  6.432288290023804
Average loss at step  7500 :  6.54589998626709
Average loss at step  8000 :  6.486287582397461
Average loss at step  8500 :  6.530690508365631
Average loss at step  9000 :  6.442089038848877
Average loss at step  9500 :  6.494717345237732
Average loss at step  10000 :  6.380370049476624
